In [6]:
import pandas as pd

data = pd.read_csv('df_for_WebApp.csv')
data

,Summary,EmbeddingsSummary,Description,EmbeddingsDescription,Comments
0,plugin config-import not found for jfrog Artif...,[ 6.66207867e-03 1.23165943e-01 -7.69825559e-...,I am following the steps from [https://docs.jf...,[ 4.73273508e-02 7.53681660e-02 -6.12716610e-...,['I am running the command provided by the doc...
1,RTDEV-5069 / mirroring https://asset-packagist...,[-8.57805684e-02 6.31687343e-02 -4.50674025e-...,We are running Artifactory 7.63.14 and have pr...,[ 3.60732339e-02 8.04025307e-02 -2.49508373e-...,"['{quote}RTDEV-5069 Fixed an issue whereby, Ar..."
2,Docker - Repository Default URL,[-2.69108848e-03 5.48766740e-02 -1.32774664e-...,Similar to Issue: [https://jfrog.atlassian.net...,[ 7.57742375e-02 7.83102438e-02 -7.85929803e-...,['This is the current value I would like to ch...
3,getting manifest unknown error pulling image,[-1.02225631e-01 -2.76588742e-02 -5.38262585e-...,I am getting the following error when pulling ...,[-6.53053671e-02 -6.07629679e-02 -1.60376308e-...,"['{\n ""schemaVersion"": 2,\n ""mediaType"": ""..."
4,Caught exception while saving incoming stream ...,[-8.13853964e-02 1.49183214e-01 -7.32800411e-...,We have recently upgrade Artifactory to 7.63.1...,[-6.47703707e-02 9.29172635e-02 -4.03810292e-...,"['""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""..."
...,...,...,...,...,...
9414,Gradle plugin should use full compilation clas...,[-3.85085270e-02 2.38638029e-01 -8.43078457e-...,[This Spring Boot 3 + Hibernate 6 sample|https...,[-2.26359032e-02 5.38617857e-02 -1.28222001e-...,['Going to need a little more information. Wh...
9415,Illegal Update-Statement generated,[-5.34606948e-02 1.21698007e-01 -1.42991794e-...,The following Entities are given\n\n{noformat}...,[-6.44817427e-02 1.06561720e-01 -6.29748497e-...,['This is due to your use of {{not-found=”igno...
9416,Trying to accomplish Insert bulk query with hi...,[ 3.79015622e-03 3.53177115e-02 -5.56239625e-...,We are working on a spring boot api which uses...,[-9.88077093e-03 2.89964397e-02 8.79406347e-...,"['', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan',..."
9417,Emulate lt/gt/le/ge tuple-comparisons when uns...,[-4.10045534e-02 -6.74348474e-02 -8.91341828e-...,Execution of the {{ ./gradlew clean test -Pdb...,[-2.17958122e-01 1.59113407e-01 -6.75906288e-...,['Thanks for the report. I see that HANA only ...


In [15]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity


model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")


def col_embed(df):
    embeddings = []

    # Iterate through each row of the column
    for i in df:
        # Encode the sentence using the SBERT model
        embedding = model.encode(i)
        # Append the embedding to the list
        embeddings.append(embedding)

    return embeddings

def sentence_embed(sentence):
    embedding = model.encode(sentence)
    return embedding


def calculate_similarity(input_embedding, embeddings):
    # input_embedding = model.encode(input_sentence)

    similarities = []
    for embedding in embeddings.tolist():
        similarity = cosine_similarity(input_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities.append(similarity)

    return similarities


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9419 entries, 0 to 9418
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Summary                9419 non-null   object
 1   EmbeddingsSummary      9419 non-null   object
 2   Description            9418 non-null   object
 3   EmbeddingsDescription  9419 non-null   object
 4   Comments               9419 non-null   object
dtypes: object(5)
memory usage: 368.1+ KB


In [10]:
data.describe()

,Summary,EmbeddingsSummary,Description,EmbeddingsDescription,Comments
count,9419,9419,9418,9419,9419
unique,9419,9417,9418,9411,8470
top,plugin config-import not found for jfrog Artif...,[-1.46329835e-01 1.25124604e-01 -9.10426583e-...,I am following the steps from [https://docs.jf...,[ 1.28439593e-03 -1.11920826e-01 -7.68679567e-...,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na..."
freq,1,2,1,2,577


In [16]:
x= 'What is the best way to get a job in data science?'
x1 = sentence_embed(x)
sum_similarity = calculate_similarity(x1, data['EmbeddingsSummary'])

AttributeError: 'str' object has no attribute 'reshape'

In [14]:
type(data['EmbeddingsSummary'][0])

str

In [ ]:
# Store summaries with similarity > 0.7 and their scores in a list
similar_summaries_and_scores = []
for summary, similarity in zip(df["Summary"], sum_similarity):
  if similarity > 0.5:
    similar_summaries_and_scores.append((summary, similarity))

# Sort similar summaries by similarity score (highest first)
sorted_similar_summaries_and_scores = sorted(similar_summaries_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (summary, similarity) in enumerate(sorted_similar_summaries_and_scores):
  if len(sum_similarity) == 0:
    print('No similar tickets found.')
  if i == 5:
    break  # Print only the top 5
  print(f"Similar summary: {summary} \n , Similarity: {similarity:.4f} \n")